## Data Cleaning and Shape Examining 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

training_data = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
testing_data = pd.read_csv("../input/test.csv", encoding="ISO-8859-1")
attribute_data = pd.read_csv('../input/attributes.csv')
descriptions = pd.read_csv('../input/product_descriptions.csv')

training_data = pd.merge(training_data, descriptions, 
                         on="product_uid", how="left")



In [ ]:
training_data.shape

As in the dataset page has mentioned that it might contains some embedded html tags, let's plot and see how many in percentage, more prececily the fields 'product_description'

In [ ]:
%matplotlib inline
total_length = len(descriptions['product_description'] )
has_tag = sum([1 for _ in descriptions['product_description'] if '<br' in _])
no_tags = total_length - has_tag

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.hist(x=[has_tag])
ax.ticklabel_format(useOffset=False)
_ = plt.xlabel('number of phrase which has html tags in')

plt.show()
print('has html tags in ',has_tag)
print('doesn\'t have html tags in ', no_tags)


Now let's see what is the frequency of search query which include digits in it with respect to product_title which 
includes words. As you can see most of them includes digits in search bar. 


In [ ]:
(training_data.search_term.str.count("\\w+") + 1).hist(bins=30) #plot number of words in search therms
(training_data.search_term.str.count("\\d+") + 1).hist(bins=30) #plot number of digits in search terms
# (training_data.product_title.str.count("\\d+") + 1).hist(bins=30)#plot number of digits in title






In [ ]:
(training_data.product_title.str.count("\\w+") + 1).hist(bins=30)#plot number of words in title
(training_data.search_term.str.count("\\w+") + 1).hist(bins=30) #plot number of words in search query





In [ ]:
(training_data.product_title.str.count("\\d+") + 1).hist(bins=30)#plot number of words in title
(training_data.search_term.str.count("\\d+") + 1).hist(bins=30) #plot number of words in search query






In [ ]:
(training_data.product_description.str.count("\\d+") + 1).hist(bins=30)
(training_data.product_description.str.count("\\d+\W+\d+") + 1).hist(bins=30)



let's plot at histogram following number of words in search query, and on the other hand relevancy score

In [ ]:
(training_data.search_term.str.count("\\w+") + 1).hist(bins=30)
(training_data.relevance + 1).hist(bins=30)



let's take a look how does the persistence of digits in the search query influence the relevancy score, from below plot it clearly that most of the search query must have between 2.0 and 3.0 

In [ ]:
(training_data.search_term.str.count("\\d+")).hist(bins=30)
(training_data.relevance ).hist(bins=30)
(training_data.search_term.str.count("^\\d+ . \\d+$")).hist(bins=30)

In [ ]:
(training_data.search_term.str.count("^\\d+ . \\d+$")).std()

let’s assume that there are zero response for null query search term


In [ ]:
training_data[training_data.search_term.str.count('\\w+') < 1]
# training_data[training_data.search_term.str.contains('^\d+') < 1]

In [ ]:
# an interest case can be see below, unfortunattly we cannot get rid of this element since it will make a bad impact on model
training_data[training_data.product_uid==100030]

In [ ]:
training_data[training_data.product_description.str.contains('.* x .*')].head(4) # at first it looks like nothing unsual 


Unfortunately, it is kind ambiguous to figure out the meaning of digits in the search context like an example below, it can mean anything. we should take care of this when cleaning context. It looks that most of the case the meaning of X is denoted the unit of measure like fit/inch/or something by something


In [ ]:
# training_data[training_data.search_term.str.contains("^\\d+ . \\d+$")].head(4)
training_data[(training_data.search_term.str.contains("^\\d+ . \\d+$") )& (training_data.relevance > 2)].head(4)

In [ ]:
# training_data[training_data.search_term.str.contains("^\\d+ . \\d+$")].head(4)
training_data[(training_data.search_term.str.contains("^\\d+ . \\d+$") )& (training_data.relevance > 2)].describe()
# (training_data.search_term.str.len() & (training_data.relevance > 2)).hist(bins=30)


In [ ]:
training_data[(training_data.search_term.str.contains("^\w+") )& (training_data.relevance > 2)].describe()


In [ ]:
# exception_number = training_data[training_data.search_term.str.contains("^\\d+ . \\d+$") ]['search_term'].values
# training_data['test'] = training_data[training_data.search_term.str.contains("^\\d+ . \\d+$") ]['search_term'].str.split(' ').values
# training_data[['product_title', 'product_description', 'relevance', ]].corr()
training_data.head(3)

In order to apply any standard method for analysis we have to standardize metric for text fields, which we define as follows

- split into tokens by white space
- remove punctuation from each token
- remove remaining tokens that are not alphabetic
- filter out stop words
- filter out short tokens

and lets also create new feature in the same time which will denote our hypothesis.

\begin{equation*}
H_1 = \{\ \frac{ card(search query)}{ card(product title)} = high\ relevance\ score\} \\
H_2 = \{ length(search query)\ influence\ relevance\ score \} \\
H_3 = \{ card(common\_words(search\ query,product\ title,product\ description)) = influence\ relevance\ score \} \\
\end{equation*}

In [ ]:
from bs4 import BeautifulSoup
import lxml
import re
import nltk
from nltk.corpus import stopwords # Import the stop word list
from nltk.metrics import edit_distance
from string import punctuation
from collections import Counter
import nltk



def remove_html_tag(text):
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text().replace('Click here to review our return policy for additional information regarding returns', '')
    return text

def str_stemmer(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return ' '.join(tokens)

def str_stemmer_title(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
    whole_set = set(str1.split())
    return sum(int(str2.find(word)>=0) for word in whole_set)

 



Now let's build feature accordingly to 
\begin{equation*}
H_1, H_2, H_3\\
\end{equation*}

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')

training_data = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
testing_data = pd.read_csv("../input/test.csv", encoding="ISO-8859-1")
attribute_data = pd.read_csv('../input/attributes.csv')
descriptions = pd.read_csv('../input/product_descriptions.csv')


num_train = training_data.shape[0] # memorize the boundary

training_data = pd.concat((training_data, testing_data), axis=0, ignore_index=True, sort=False)
training_data = pd.merge(training_data, descriptions, how='left', on='product_uid')




In [ ]:
############### cleaning html tags ##################
has_tag_in = training_data.product_description.str.contains('<br')
training_data.loc[has_tag_in, 'product_description'] = training_data.loc[has_tag_in, 'product_description'].map(lambda x:remove_html_tag(x))
###############

############## apply stemming #####################
training_data['search_term'] = training_data['search_term'].map(lambda x:str_stemmer_title(x))
training_data['product_title'] = training_data['product_title'].map(lambda x:str_stemmer(x))

training_data['product_description'] = training_data['product_description'].map(lambda x:str_stemmer(x))
############## end stemming #####################

############## building custome feature, let's build a few of them before compare which one is the best ###########
training_data['len_of_query'] = training_data['search_term'].map(lambda x:len(x.split())).astype(np.int64)
training_data['shared_words'] = training_data[['search_term','product_description', 'product_title']].apply(lambda row:sum([str_common_word(*row[:-1]), str_common_word(*row[1:])]), axis=1)

# training_data['frequency_digits_in_sq']=training_data.product_description.str.count("\\d+")
training_data['frequency_words_in_sq'] = training_data.product_description.str.count("\\w+")
training_data["distance"] = training_data.loc[:, ["search_term","product_title"]].apply(lambda x: edit_distance(*x), axis=1)


In [ ]:
#let's take a look if there is not empty search query now
# empty_search_query = training_data[training_data.search_term.str.count('\\w+') < 1].values
# print('data frame of empty seach query along with products',empty_search_query)
# # training_data[training_data.product_uid==100030]
is_anything_none = training_data.isnull().values.any()
print('presence of Nan values',  is_anything_none)


In [ ]:
training_data[training_data.search_term.str.match("\\d+ x \\d+") > 0]

In [ ]:
training_data[['product_title','len_of_query','shared_words','frequency_words_in_sq','relevance']].corr()

In [ ]:
df_all = training_data.drop(['search_term','product_description','product_title'],axis=1)


df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']

y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

rf = RandomForestRegressor(n_estimators=4, max_depth=6, random_state=0)
clf = BaggingRegressor(rf, n_estimators=4, max_samples=0.1, random_state=25)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)


Analysis of the Model


In [ ]:
# from nltk.metrics import edit_distance
# from sklearn.preprocessing import StandardScaler

# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer('english')


# df_train = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
# df_test = pd.read_csv("../input/test.csv", encoding="ISO-8859-1")
# attribute_data = pd.read_csv('../input/attributes.csv')
# df_pro_desc = pd.read_csv('../input/product_descriptions.csv')



# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# num_train = df_train.shape[0]

# ###############
# from bs4 import BeautifulSoup
# import lxml
# import re
# import nltk
# from nltk.corpus import stopwords # Import the stop word list
# from nltk.metrics import edit_distance
# from string import punctuation
# from collections import Counter
# import nltk


# def remove_html_tag(text):
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text().replace('Click here to review our return policy for additional information regarding returns', '')
#     return text

# def str_stemmer(doc):
#     # split into tokens by white space
#     tokens = doc.split()
#     # remove punctuation from each token
#     table = str.maketrans('', '', punctuation)
#     tokens = [w.translate(table) for w in tokens]
#     # remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#     # filter out stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [w for w in tokens if not w in stop_words]
#     # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     return ' '.join(tokens)

# def str_stemmer_title(s):
#     return " ".join([stemmer.stem(word) for word in s.lower().split()])

# def str_common_word(str1, str2):
#     whole_set = set(str1.split())
#     return sum(int(str2.find(word)>=0) for word in whole_set)

 




# ###############
# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')


# ############## apply stemming #####################
# df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer_title(x))
# df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))

# df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))
# ############## end stemming #####################

# ############## building custome feature, let's build a few of them before compare which one is the best ###########
# df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
# df_all['shared_words'] = df_all[['search_term','product_description', 'product_title']].apply(lambda row:sum([str_common_word(*row[:-1]), str_common_word(*row[1:])]), axis=1).astype(np.int64)

# # training_data['frequency_digits_in_sq']=training_data.product_description.str.count("\\d+")
# df_all['frequency_words_in_sq'] = df_all.product_description.str.count("\\w+").astype(np.int64)
# df_all["distance_levistein"] = df_all.loc[:, ["search_term","product_title"]].apply(lambda x: edit_distance(*x), axis=1).astype(np.int64)

# df_all['length in product info'] = training_data[['product_title','product_description']].apply(lambda row:sum([len(*row[:-1]), len(*row[1:])]), axis=1).astype(np.int64)

# df_all = df_all.drop(['search_term','product_title','product_description'],axis=1)


# df_train = df_all.iloc[:num_train]
# print('df_train',df_train)
# df_test = df_all.iloc[num_train:]
# print('df_test',df_test)
# id_test = df_test['id']

# y_train = df_train['relevance'].values
# X_train = df_train.drop(['id','relevance'],axis=1).values
# X_test = df_test.drop(['id','relevance'],axis=1).values

# #### Feature to the same scale
# scX = StandardScaler()
# X_train = scX.fit_transform(X_train)
# X_test = scX.fit_transform(X_test)

# rf = RandomForestRegressor(n_estimators=4, max_depth=6, random_state=0)
# clf = BaggingRegressor(rf, n_estimators=4, max_samples=0.1, random_state=25)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

# pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

In [ ]:
training_data[training_data.search_term.str.count('\\w+')<1]

In [ ]:
# df[training_data.isnull]

In [ ]:
# from nltk.metrics import edit_distance
# from sklearn.preprocessing import StandardScaler

# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer('english')


# df_train = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
# df_test = pd.read_csv("../input/test.csv", encoding="ISO-8859-1")
# attribute_data = pd.read_csv('../input/attributes.csv')
# df_pro_desc = pd.read_csv('../input/product_descriptions.csv')



# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# num_train = df_train.shape[0]

# ###############
# from bs4 import BeautifulSoup
# import lxml
# import re
# import nltk
# from nltk.corpus import stopwords # Import the stop word list
# from nltk.metrics import edit_distance
# from string import punctuation
# from collections import Counter
# import nltk


# def remove_html_tag(text):
#     soup = BeautifulSoup(text, 'lxml')
#     text = soup.get_text().replace('Click here to review our return policy for additional information regarding returns', '')
#     return text

# def str_stemmer(doc):
#     # split into tokens by white space
#     tokens = doc.split()
#     # remove punctuation from each token
#     table = str.maketrans('', '', punctuation)
#     tokens = [w.translate(table) for w in tokens]
#     # remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#     # filter out stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [w for w in tokens if not w in stop_words]
#     # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     return ' '.join(tokens)

# def str_stemmer_title(s):
#     return " ".join([stemmer.stem(word) for word in s.lower().split()])

# def str_common_word(str1, str2):
#     whole_set = set(str1.split())
#     return sum(int(str2.find(word)>=0) for word in whole_set)



# ###############
# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')



Cleaning function

In [ ]:
# from nltk.corpus import brown, stopwords
# from nltk.cluster.util import cosine_distance
 
# def sentence_similarity(sent1, sent2, stopwords=None):
#     sent1 = sent1.split(' ')
#     sent2 = sent2.split(' ')
#     if stopwords is None:
#         stopwords = []
 
#     sent1 = [w.lower() for w in sent1]
#     sent2 = [w.lower() for w in sent2]
 
#     all_words = list(set(sent1 + sent2))
 
#     vector1 = [0] * len(all_words)
#     vector2 = [0] * len(all_words)
 
#     # build the vector for the first sentence
#     for w in sent1:
#         if w in stopwords:
#             continue
#         vector1[all_words.index(w)] += 1
 
#     # build the vector for the second sentence
#     for w in sent2:
#         if w in stopwords:
#             continue
#         vector2[all_words.index(w)] += 1
 
#     return 1 - cosine_distance(vector1, vector2)

In [ ]:

# ############## apply stemming #####################
# df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer_title(x))
# df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))

# df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))
# ############## end stemming #####################

# ############## building custome feature, let's build a few of them before compare which one is the best ###########
# df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
# df_all['shared_words'] = df_all[['search_term','product_description', 'product_title']].apply(lambda row:sum([str_common_word(*row[:-1]), str_common_word(*row[1:])]), axis=1).astype(np.int64)

# # training_data['frequency_digits_in_sq']=training_data.product_description.str.count("\\d+")
# df_all['frequency_words_in_sq'] = df_all.product_description.str.count("\\w+").astype(np.int64)
# df_all["distance_levistein"] = df_all.loc[:, ["search_term","product_title"]].apply(lambda x: edit_distance(*x), axis=1).astype(np.int64)

# df_all['length in product info'] = training_data[['product_title','product_description']].apply(lambda row:sum([len(*row[:-1]), len(*row[1:])]), axis=1).astype(np.int64)

# df_all = df_all.drop(['search_term','product_title','product_description'],axis=1)


# # df_train = df_all.iloc[:num_train]
# # print('df_train',df_train)
# # df_test = df_all.iloc[num_train:]
# # print('df_test',df_test)
# # id_test = df_test['id']

# # y_train = df_train['relevance'].values
# # X_train = df_train.drop(['id','relevance'],axis=1).values
# # X_test = df_test.drop(['id','relevance'],axis=1).values

# # #### Feature to the same scale
# # scX = StandardScaler()
# # X_train = scX.fit_transform(X_train)
# # X_test = scX.fit_transform(X_test)

# # rf = RandomForestRegressor(n_estimators=4, max_depth=6, random_state=0)
# # clf = BaggingRegressor(rf, n_estimators=4, max_samples=0.1, random_state=25)
# # clf.fit(X_train, y_train)
# # y_pred = clf.predict(X_test)

# # pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

In [ ]:
# from nltk.metrics import edit_distance
# from sklearn.preprocessing import StandardScaler

# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
# from nltk.stem.snowball import SnowballStemmer

# stemmer = SnowballStemmer('english')


# df_train = pd.read_csv("../input/train.csv", encoding="ISO-8859-1")
# df_test = pd.read_csv("../input/test.csv", encoding="ISO-8859-1")
# attribute_data = pd.read_csv('../input/attributes.csv')
# df_pro_desc = pd.read_csv('../input/product_descriptions.csv')



# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# num_train = df_train.shape[0]

# # def str_stemmer(s):
# #     return " ".join([stemmer.stem(word) for word in s.lower().split()])

# def str_common_word(str1, str2):
#     return sum(int(str2.find(word)>=0) for word in str1.split())


# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

# df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')



